In [ ]:
import os

# Change current working directory to parent directory
# Save the original working directory
original_cwd = os.getcwd()

# Change to parent directory
os.chdir("..")


from tools import *
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
tripadvisor_api_key = os.getenv("TRIPADVISOR_API_KEY")
rapid_api_key = os.getenv("RAPID_API_KEY_ALT_6")


In [ ]:
# url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"

# querystring = {"query":"new york","page":"1"}

# headers = {
# 	"x-rapidapi-key": "b2b95a6827msha1cb6f6f392e51cp1893f6jsnd6b152f989ba",
# 	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
# }

# response = requests.get(url, headers=headers, params=querystring)

# print(response.json())

{'link': 'https://www.tripadvisor.com/Restaurants-g60763-New_York_City_New_York.html', 'total_pages': 333, 'current_page': 1, 'total_items_count': 10000, 'items_count': 35, 'results': [{'id': 27967019, 'name': 'Bustronome New York', 'link': 'https://www.tripadvisor.com/Restaurant_Review-g60763-d27967019-Reviews-Bustronome_New_York-New_York_City_New_York.html', 'reviews': 92, 'rating': 4.8, 'price_range_usd': '$$$$', 'is_sponsored': True, 'menu_link': 'https://bustronome.com/en/new-york/menu/', 'reservation_link': 'https://tripadvisor.com/Commerce?p=OpenTable&src=261568826&geo=27967019&from=Restaurants&area=reservation_button&slot=1&matchID=1&oos=0&cnt=1&silo=45582&bucket=979641&nrank=1&crank=1&clt=R&ttype=Restaurant&tm=330062411&managed=false&capped=false&gosox=9DMvibDTWdNaFeGYL8QTakyv1LBYZaGpeB4MEHn-hv2mZBN376EHVCGusTIRxmJ8jAjAU8_t8WEJpk9vaRXt-bCnr3Jwyqb6HlUemd3scRU&cs=137d5b31f52dd2d6d967ecb6961b9e569_M2M', 'featured_image': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d

In [2]:
import json

# Filepath to the JSON file
file_path = "data/vietnam-provinces.json"

# Open and read the JSON file
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

provinces = [data[i]['NameEn'] for i in range(len(data))]

In [3]:
provinces[0]

'Ha Noi'

In [10]:
# import requests

# url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"

# querystring = {"query":'Vung Tau',"page":"2"}

# headers = {
# 	"x-rapidapi-key": "f01c3f6968msh4a12cf8c0c8f3c2p1d9e36jsne6d66dae12ae",
# 	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
# }

# response = requests.get(url, headers=headers, params=querystring)

In [11]:
# response.raise_for_status()  # Raise an error for bad responses
# data = response.json()
# data

In [12]:
# data['results'][0]

In [ ]:
import csv

# Assuming 'data' is already fetched as in your previous cells
results = data['results']

# Get the keys for CSV columns from the first result
if results:
    columns = results[0].keys()
    csv_filename = f"{provinces[1]}.csv"
    # Create a CSV file with the province name
    with open(csv_filename, "w", newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()
        writer.writerows(results)
    print(f"Saved {len(results)} rows to {csv_filename}")
else:
    print("No results found.")

Saved 30 rows to Ha Giang.csv


In [4]:
import os
import json
import requests
import pandas as pd

def save_all_restaurants_to_csv(api_key, province_name):
    """
    Fetches all restaurant data for a given province from the TripAdvisor Scraper API
    and saves it to a CSV file.

    Args:
        api_key (str): Your RapidAPI key for the TripAdvisor Scraper API.
        province_name (str): The name of the province to search for restaurants.
    """
    url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"
    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
    }
    all_results = []

    # --- Get the first page to determine the total number of pages ---
    print(f"Fetching initial data for {province_name} to determine total pages...")
    querystring = {"query": province_name, "page": 1}
    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        data = response.json()
        total_pages = data.get("total_pages", 1)
        print(f"Found {total_pages} pages of results for {province_name}.")
        # Add the results from the first page
        all_results.extend(data.get("results", []))
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the first page for {province_name}: {e}")
        return # Exit the function if the first request fails

    # --- Collect results from all subsequent pages ---
    # Start from page 2 since we already fetched page 1
    for page_number in range(2, total_pages + 1):
        querystring = {"query": province_name, "page": page_number}
        try:
            response = requests.get(url, headers=headers, params=querystring)
            response.raise_for_status()
            page_data = response.json()
            results = page_data.get("results", [])
            all_results.extend(results)
            print(f"Fetched page {page_number}/{total_pages} for {province_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page_number} for {province_name}: {e}")
            # Continue to the next page even if one fails
            continue

    # --- Convert to DataFrame and save as CSV ---
    if all_results:
        print("Converting data to DataFrame and saving to CSV...")
        df = pd.DataFrame(all_results)

        # Set 'restaurant_id' as the index if it exists
        if "restaurant_id" in df.columns:
            df.set_index("restaurant_id", inplace=True)

        # Create the directory if it doesn't exist
        output_dir = os.path.join("data", "rapid_restaurant_id")
        os.makedirs(output_dir, exist_ok=True)

        # Define the output path and save the CSV
        output_path = os.path.join(output_dir, f"{province_name}.csv")
        df.to_csv(output_path, encoding="utf-8-sig")
        print(f"✅ Successfully saved all restaurants for {province_name} to {output_path}")
    else:
        print(f"No results were found for {province_name}.")



In [2]:
PROVINCE = 'Vung Tau'


In [8]:
os.chdir(original_cwd)
save_all_restaurants_to_csv('b2b95a6827msha1cb6f6f392e51cp1893f6jsnd6b152f989ba', PROVINCE)

Fetching initial data for Vung Tau to determine total pages...
Found 8 pages of results for Vung Tau.
Fetched page 2/8 for Vung Tau
Fetched page 3/8 for Vung Tau
Fetched page 4/8 for Vung Tau
Fetched page 5/8 for Vung Tau
Fetched page 6/8 for Vung Tau
Fetched page 7/8 for Vung Tau
Fetched page 8/8 for Vung Tau
Converting data to DataFrame and saving to CSV...
✅ Successfully saved all restaurants for Vung Tau to data\rapid_restaurant_id\Vung Tau.csv
